In [23]:
from __future__ import division
import os
import re
import random
import time
import binascii
from bisect import bisect_right
from heapq import heappop, heappush

from os import listdir
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [25]:
# Read dataset
sheet = pd.DataFrame()
for csv_file in listdir('./150k/'):
    sheet = sheet.append(pd.read_csv(f'./150k/{csv_file}'))

df = pd.DataFrame(sheet['content'])
# limit data size
df = df[:2000]

# perform some cleaning
df['content'] = df['content'].apply(lambda x: x.replace('\n',''))
df['content'] = df['content'].apply(lambda x: BeautifulSoup(x, features="html.parser").get_text())
df.shape[0]

2000

In [26]:
# Calculate Shingles
print("Shingling articles...")

doc_shingle_sets = []

t0 = time.time()
totalShingles = 0

for doc in df['content']:

    # 'shinglesInDoc' will hold all of the unique shingle IDs present in the 
    # current document. If a shingle ID occurs multiple times in the document,
    # it will only appear once in the set (this is a property of Python sets).
    shinglesInDoc = set()
    
    words = doc.split(" ")
    
    # For each word in the document...
    for index in range(0, len(words) - 2):
        # Construct the shingle text by combining three words together.
        shingle = words[index] + " " + words[index + 1] + " " + words[index + 2]

        # Hash the shingle to a 32-bit integer.
        crc = binascii.crc32(shingle.encode('utf-8')) & 0xffffffff
    
        # Add the hash value to the list of shingles for the current document. 
        # Note that set objects will only add the value to the set if the set 
        # doesn't already contain it. 
        shinglesInDoc.add(crc)
    
    # store shingle set
    doc_shingle_sets.append(shinglesInDoc)
    totalShingles = totalShingles + (len(words) - 2)

# Report how long shingling took.
print(f"shingling {df.shape[0]} docs took {time.time() - t0} sec.") 
print(f"average shingles per doc: {totalShingles / df.shape[0]}")


Shingling articles...
shingling 2000 docs took 1.0387790203094482 sec.
average shingles per doc: 387.412


In [27]:
numDocs = df.shape[0]

In [28]:
# Generate MinHash Signatures
print('Generating random hash functions...')

# Our random hash function will take the form of:
# h(x) = (a*x + b) % c
# Where 'x' is the input value, 'a' and 'b' are random coefficients, and 'c' is
# a prime number just greater than maxShingleID.

# This is the number of components in the resulting MinHash signatures.
# Correspondingly, it is also the number of random hash functions that
# we will need in order to calculate the MinHash.
numHashes = 10;

# Record the maximum shingle ID that we assigned.
maxShingleID = 2**32-1

# We need the next largest prime number above 'maxShingleID'.
nextPrime = 4294967311

def pickRandomCoeffs(k):
    # Create a list of 'k' random values.
    randList = []
    while k > 0:
        # Get a random shingle ID.
        randIndex = random.randint(0, maxShingleID) 
        # Ensure that each random number is unique.
        while randIndex in randList:
            randIndex = random.randint(0, maxShingleID) 
        # Add the random number to the list.
        randList.append(randIndex)
        k = k - 1
    return randList

coeffA = pickRandomCoeffs(numHashes)
coeffB = pickRandomCoeffs(numHashes)

Generating random hash functions...


In [29]:
print('Generating MinHash signatures for all documents...')

t0 = time.time()

# List of documents represented as signature vectors
signatures = []

for i in range(numDocs):
    
    # Get the shingle set for this document.
    shingle_set = doc_shingle_sets[i]
    
    # The resulting minhash signature for this document. 
    signature = []
    
    for i in range(0, numHashes):
        minHashCode = nextPrime + 1
        
        for shingle in shingle_set:
            # Evaluate the hash function.
            hashCode = (coeffA[i] * shingle + coeffB[i]) % nextPrime
            
            # Track the lowest hash code seen.
            if hashCode < minHashCode:
                minHashCode = hashCode
        
        # store doc signature
        signature.append(minHashCode)
        
    # Store the MinHash signature for this document.
    signatures.append(signature)
    
print(f"Generating MinHash signatures took {time.time() - t0} sec")

Generating MinHash signatures for all documents...
Generating MinHash signatures took 2.880990982055664 sec


In [30]:
# Compare All Signatures
print('Comparing all signatures...')

similar_docs = []

jac_threshold = 0.5

t0 = time.time()

for i in range(numDocs):
    
    similar_to_i = []
    
    # Get the MinHash signature for document i.
    signature1 = signatures[i]
    
    for j in range(i + 1, numDocs):
    
        # Get the MinHash signature for document j.
        signature2 = signatures[j]
    
        count = 0
        # Count the number of positions in the minhash signature which are equal.
        for k in range(0, numHashes):
            count = count + (signature1[k] == signature2[k])
    
        # Record the percentage of positions which matched.
        estJSim = (count / numHashes)
    
        if estJSim > jac_threshold:
            similar_to_i.append(j)
    
    similar_docs.append(similar_to_i)
        
print(f"Comparing MinHash signatures took {time.time() - t0} sec")

Comparing all signatures...
Comparing MinHash signatures took 4.73025107383728 sec


In [16]:
# Aggregate Data
# final = pd.DataFrame(columns=['content','similar_docs'])
for i in range(numDocs):
    sim_to_i = similar_docs[i]
    res = ''
    for j in sim_to_i:
        res += str(j) + ","
    print(f"similars to {i} : {res}")

similars to 0 : 
similars to 1 : 
similars to 2 : 
similars to 3 : 
similars to 4 : 
similars to 5 : 
similars to 6 : 
similars to 7 : 
similars to 8 : 26,105,115,766,999,1719,1888,1974,
similars to 9 : 
similars to 10 : 1006,1023,1475,
similars to 11 : 
similars to 12 : 
similars to 13 : 
similars to 14 : 
similars to 15 : 
similars to 16 : 
similars to 17 : 
similars to 18 : 
similars to 19 : 
similars to 20 : 
similars to 21 : 
similars to 22 : 
similars to 23 : 
similars to 24 : 
similars to 25 : 
similars to 26 : 105,115,766,999,1719,1888,1974,
similars to 27 : 
similars to 28 : 
similars to 29 : 
similars to 30 : 
similars to 31 : 
similars to 32 : 
similars to 33 : 1960,
similars to 34 : 934,
similars to 35 : 
similars to 36 : 
similars to 37 : 
similars to 38 : 
similars to 39 : 
similars to 40 : 
similars to 41 : 
similars to 42 : 
similars to 43 : 
similars to 44 : 
similars to 45 : 275,1347,
similars to 46 : 
similars to 47 : 
similars to 48 : 
similars to 49 : 
similars to 

similars to 1353 : 
similars to 1354 : 
similars to 1355 : 1410,
similars to 1356 : 
similars to 1357 : 
similars to 1358 : 
similars to 1359 : 
similars to 1360 : 
similars to 1361 : 
similars to 1362 : 
similars to 1363 : 
similars to 1364 : 
similars to 1365 : 
similars to 1366 : 
similars to 1367 : 
similars to 1368 : 1387,1452,1671,1995,
similars to 1369 : 
similars to 1370 : 
similars to 1371 : 
similars to 1372 : 
similars to 1373 : 
similars to 1374 : 
similars to 1375 : 
similars to 1376 : 
similars to 1377 : 
similars to 1378 : 
similars to 1379 : 
similars to 1380 : 
similars to 1381 : 
similars to 1382 : 
similars to 1383 : 
similars to 1384 : 
similars to 1385 : 
similars to 1386 : 
similars to 1387 : 1449,1452,1475,1633,1671,1732,1802,1828,1876,1995,
similars to 1388 : 
similars to 1389 : 
similars to 1390 : 
similars to 1391 : 
similars to 1392 : 
similars to 1393 : 
similars to 1394 : 
similars to 1395 : 
similars to 1396 : 
similars to 1397 : 
similars to 1398 : 
simil